In [1]:
import ps, ps_utils
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
ps_utils.switch_to_prod_pvlss()
no_skew_model = 'HybridLGM1FDupireLocalVolatilityModel'
skew_model = 'HybridCheyette1FactorLocalVolatilityModel'
import warnings
warnings.filterwarnings("ignore") 

from ps_utils.env import ps
import numpy as np
from ps_utils import env, market, analytics, pricing
import pandas as pd

from typing import Union, Optional, Dict, List
PsObject = Union[ps.Message, ps.GlobalID]


ps_utils.global_algo_id()

[ERROR] 2023-07-03 15:17:53,132 - You do not have xbbg installed, some functionality will not be available
[INFO] 2023-07-03 15:17:53,882 - Nothing to do: current environment is already prod_pvlss


'6c21bbf749e70ca1d53f365a33d8b184691c53c2'

In [2]:
def get_df(response_hashid, isPrev):
    r = ps.get(response_hashid)
    mas_cpty=[]
    mas_resp_hash=[]
    mas_bcva=[]
    for i,el in enumerate(r.Result):
        mas_cpty.append(ps.get(el._nonce)['_Counterparty'])
        mas_resp_hash.append(el._id)
        try:
            mas_bcva.append(el.Result['BCVA'])
        except:
            mas_bcva.append(np.nan)
    df=pd.DataFrame({'Cpty':mas_cpty, 'Resp_Hash_tek':mas_resp_hash,'BCVA_tek':mas_bcva})
    if isPrev:
        df.rename(columns={'Resp_Hash_tek':'Resp_Hash_prev','BCVA_tek':'BCVA_prev'},inplace=True)
    return df

resp_prev=ps.get('e32436c7fea7522996787a6f66670afb91147792')
resp_tek=ps.get('7ac40ba63ecb042f46d138d3ad8133668e1b20a1')

dtprev = resp_prev['Result'][0]['Request']['Model']['MarketDataSet']['AsOfDate'].date().strftime("%Y-%m-%d")
dttek = resp_tek['Result'][0]['Request']['Model']['MarketDataSet']['AsOfDate'].date().strftime("%Y-%m-%d")

df_prev=get_df(resp_prev, isPrev=True)
df_tek=get_df(resp_tek, isPrev=False)
df=df_prev.merge(df_tek, on='Cpty', how='outer')

newoldcpties=pd.read_excel('OldNewCpties.xlsx')
newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
df=df.merge(newoldcpties,on='Cpty',how='left')

print("PrevBCVA=",df_prev['BCVA_prev'].sum())
print("TekBCVA=",df_tek['BCVA_tek'].sum())
print("diff=",df_tek['BCVA_tek'].sum()-df_prev['BCVA_prev'].sum())

#mask=pd.read_excel('tmp_Names30Cpties.xlsx')['Cpty'].values
#mask=list(map(lambda x: x in mask, df['Cpty']))
#dfmasked=df[mask]
#dfmasked['BCVA_diff']=dfmasked['BCVA_tek']-dfmasked['BCVA_prev']
#dfmasked.head(5)

mask1na=df['BCVA_prev'].isna()
mask2na=df['BCVA_tek'].isna()

# print("New cpties:",df[mask1na]['BCVA_tek'].sum(), df[mask1na]['Cpty'].unique())
# print("Expired cpties:",df[mask2na]['BCVA_prev'].sum(),df[mask2na]['Cpty'].unique())

new_cpties=df[mask1na]
expired_cpties=df[mask2na]
print("New cpties old book:",new_cpties[new_cpties['Old/New']=='Old']['BCVA_tek'].sum(), new_cpties[new_cpties['Old/New']=='Old']['Cpty'].unique())
print("Expired cpties old book:",expired_cpties[expired_cpties['Old/New']=='Old']['BCVA_tek'].sum(), expired_cpties[expired_cpties['Old/New']=='Old']['Cpty'].unique())
print("New cpties new book:",new_cpties[new_cpties['Old/New']=='New']['BCVA_tek'].sum(), new_cpties[new_cpties['Old/New']=='New']['Cpty'].unique())
print("Expired cpties new book:",expired_cpties[expired_cpties['Old/New']=='New']['BCVA_tek'].sum(), expired_cpties[expired_cpties['Old/New']=='New']['Cpty'].unique())


maskna=~(mask1na^mask2na)
dfmasked=df
#dfmasked=df[maskna]
#dfmasked.reset_index()
dfmasked['abs_old_BCVA']=np.abs(dfmasked['BCVA_prev'])
dfmasked['diff_BCVA']=dfmasked['BCVA_tek']-dfmasked['BCVA_prev']
dfmasked['abs_diff_BCVA']=np.abs(dfmasked['diff_BCVA'])
#dfmasked=dfmasked[dfmasked['abs_old_BCVA']>0].reset_index(drop=True)

# newoldcpties=pd.read_excel('OldNewCpties.xlsx')
# newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
# dfmasked=dfmasked.merge(newoldcpties,on='Cpty',how='left')
dfmasked_old=dfmasked[dfmasked['Old/New']=='Old'].reset_index(drop=True)
dfmasked_new=dfmasked[dfmasked['Old/New']=='New'].reset_index(drop=True)
print("AmtCpties to run pnl explain:",len(dfmasked))
print("Total diff masked=",dfmasked['diff_BCVA'].sum())
print("Old book diff masked=",dfmasked_old['diff_BCVA'].sum())
print("New book diff masked=",dfmasked_new['diff_BCVA'].sum())
dfmasked.sort_values(by='abs_diff_BCVA', ascending=False).head()

PrevBCVA= -15535754.216342846
TekBCVA= -16834206.219399303
diff= -1298452.0030564573
New cpties old book: -692.5560564246293 ['KPMAL' 'SARST' 'VTMEK']
Expired cpties old book: 0.0 []
New cpties new book: -158073.74323213787 ['VBBBB' 'DRUZA' 'LUBOS' 'SAMZE' 'KKKKS' 'SBKRR' 'SBTKB' 'SZNOV' 'FRMMK'
 'MEDUN' 'MBVVL' 'IMLEN' 'STREI' 'TKSSS' 'PPPPR' 'TEHUG' 'DVAMR' 'SZVDK'
 'NIKKA' '1AAMB' 'ZARPO' 'VVVBB' 'TESVE' 'PRMTP' 'KOARS' '1AAOH' 'NNNNH'
 'DVBRR' 'RUSAN' 'GKSOK' 'NNNNV' 'CBTTT' 'RPBUN' 'UUBBB' 'MEBZZ' 'NIZZZ'
 'ARHZZ' 'PVBSM' 'VEDER' 'METRA' 'SBBKO' 'MULTT' 'CCCBR' 'ZZZAD' 'DVBVV'
 'VINOO' 'VECHB' 'TISHA' 'AVALL' '1AAPC' 'VETMG' 'SZZVE' 'KARTR' 'TSDEV'
 'MLRDD' 'BAYDM' 'SZASS' '1AAIA' 'PVBZZ' 'ALFOR' 'SEVZZ' 'PRKIM' 'VVBBB'
 '1AAMT' 'KOMME' 'UZBSS' 'AGRRB' 'STIUN' 'PROKC' 'OMGBG' 'AROMR' 'MKMIH'
 'MOBLL' '1AAHU' 'RUUTT' 'GOLAH' 'SFTTR' 'SLAKO' 'SENDO' 'AGRUR' 'UZBRR'
 'NNPPP' 'DDDDP' 'TDVKT' 'VOLEG' 'AHNIZ' 'ROSHH' 'PARKZ' 'TPKSA' 'SSBBT'
 'BBSSS' 'KRACN' 'MGSTT' 'BBZTM' 'SZBNK' 'MSSS

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
148,MBKRK,2a4192b5e015b51922a5e933fe2acf6a471566a9,"-76,291.18",3b65536b9e6199fc234d2cff6a10277329347d9a,"-754,693.07",New,BCVA,"76,291.18","-678,401.88","678,401.88"
905,SZSTP,e5ddfaa22c3b1b4ad616a9132b08878781ffcfa5,"-238,988.04",323ad21a867f2ad9d5f80e287fc0ba64109d7b1a,"-602,618.38",New,BCVA,"238,988.04","-363,630.33","363,630.33"
24,VNEKB,7a2628c146b6e80a40df7138fc9b3f84c8514c45,"1,136,744.64",a3fecb7990fccdfa6835daccc5aeed747e05a108,"1,395,366.67",Old,BCVA,"1,136,744.64","258,622.02","258,622.02"
1332,SIBRH,a1e5e9b110fe3b524183ead7001a93831452b83d,"-3,987,017.71",723e0e37ae76b3bcf70dc0da7a1217578946f557,"-4,153,685.13",New,BCVA,"3,987,017.71","-166,667.42","166,667.42"
622,GRIDD,5bef469f6f853db80f2ca1525109618072899d27,"-617,309.69",621068af3ba71981333cde25514e402060e563e2,"-763,577.64",New,BCVA,"617,309.69","-146,267.95","146,267.95"


In [3]:
dfmasked[dfmasked['Old/New'].isna()]

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
46,FOTON,7ae6da71d0319c420c40512c49dfd5d92c50260a,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN
54,EVKOK,91d49264d65f01f5b2226d1dcb1c3d35ffebf5bb,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN
130,PTROL,23470201a664482c3b9512f3913574c8c99755a7,NaN,2ca5f7284132b179895b4e54a5bbccb6563d5cca,NaN,NaN,NaN,NaN,NaN,NaN
156,FSKSS,74eb94743c82645733b888619da15768dc1a46cb,NaN,21b104ff94217758fc2a0c39c931b72dcd8c2af6,NaN,NaN,NaN,NaN,NaN,NaN
254,MBDDR,1d7d6ec66c4f897d3341e7511db44c7710af6f45,NaN,72110ae3b9d0e6a35985661c4d68a7f8b5a7e2df,NaN,NaN,NaN,NaN,NaN,NaN
551,ARNEV,5df211c47188c2f7cfc73e79f245071e03c7fed6,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN
584,KEVRP,87c34a0303d9f7024ea4a67b20a28c672f3c9eed,8.41,NaN,NaN,NaN,NaN,8.41,NaN,NaN
601,URBEN,02921cec135c9d5b7eee756fc55e862f47ec9dd0,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN
671,RKOOO,27ff9483d75abe901ec98a1a4d359f39a11e6175,0.01,NaN,NaN,NaN,NaN,0.01,NaN,NaN
708,FUDSS,17f038d850c78382de4e400ad56b78c7a3b177b9,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN


In [6]:
print("all=",dfmasked['BCVA_prev'].sum())
print("old=",dfmasked_old['BCVA_prev'].sum())
print("new=",dfmasked_new['BCVA_prev'].sum())
print("old+new=",dfmasked_old['BCVA_prev'].sum()+dfmasked_new['BCVA_prev'].sum())

all= -16287112.087532878
old= -6015092.59075744
new= -10272019.496775437
old+new= -16287112.087532878


In [12]:
dfmasked[dfmasked['Cpty']=='SZKOT']

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
119,SZKOT,aa09da4efbba09f7db32eb1cf79608006a669f7e,"-5,641.53",c1b7268b576092d082172e479655c1d075d5b0d0,"-6,513.62",New,BCVA,"5,641.53",-872.09,872.09


In [15]:
dfmasked[dfmasked['Cpty']=='FNKOP']

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
446,FNKOP,b31f1e27e0def47d7567ee0ee9b0cb77a0fac6c1,"22,363.95",912a5c445c132f80d7559c4d115aeebd5175d73f,"3,792.05",New,BCVA,"22,363.95","-18,571.90","18,571.90"


In [4]:
algo_id = '833aa9286a2951a0eac3fb38913d6990435e5c4c'


reqv_tek=ps.get(resp_tek._nonce)
reqv_prev=ps.get(resp_prev._nonce)


queries = ['BCVA']
big_request = ps_utils.prepare_big_request_for_pnl(reqv_tek, reqv_prev, algo_id,'USD', queries)

b = ps_utils.run_generic_revaluation_based_metric_change_explain_batch(big_request, algo_id, True)

ps.put(b)
ps_utils.env.logger.info(f'Result id is {b._id}')

# df = pd.DataFrame(columns=['MetricType', 'Category', 'Counterparty', 'Value'])
# for result in a:
#     for metric in queries:
#         for category in result[metric]:
#             df = pd.concat([df, pd.DataFrame([{
#                 'MetricType': metric,
#                 'Category': category,
#                 'Counterparty': result.Request['_Counterparty'],
#                 'Value': result[metric][category]}])])

# df.to_excel('PnLExplain.xlsx', index=False)

[INFO] 2023-07-03 15:29:12,637 - Removing failed cpties
[INFO] 2023-07-03 15:29:48,164 - Cpties removed
[INFO] 2023-07-03 15:29:48,165 - PNL Explain started
[INFO] 2023-07-03 15:29:48,166 - Doing some preparations
[INFO] 2023-07-03 15:29:48,167 - Converting market data to canonical format
[INFO] 2023-07-03 15:29:48,712 - Converting done
[INFO] 2023-07-03 15:29:48,714 - Collecting dependencies
[INFO] 2023-07-03 15:45:32,773 - Market dependencies collected
[INFO] 2023-07-03 15:45:32,774 - Preparations done
[INFO] 2023-07-03 15:45:32,775 - Calculating Total
[INFO] 2023-07-03 15:48:59,242 - Total calculated
[INFO] 2023-07-03 15:48:59,243 - Calculating StaticDataSet change effect
[INFO] 2023-07-03 15:48:59,272 - StaticDataSet change not needed
[INFO] 2023-07-03 15:48:59,273 - Calculating Product change effect
[INFO] 2023-07-03 15:49:21,663 - Product change calculated
[INFO] 2023-07-03 15:49:21,665 - Model change effect calculation
[INFO] 2023-07-03 15:49:21,666 - NumeraireCurrency change ef

[INFO] 2023-07-03 16:09:34,521 - Processing RatesCurvesBundles.CNH_SOFR done
[INFO] 2023-07-03 16:09:34,523 - Processing RatesCurvesBundles.EUR_EURIBOR_6M
[INFO] 2023-07-03 16:09:34,524 - Collecting requests
[INFO] 2023-07-03 16:09:38,307 - Requests collected. Request=05da2f7051205ca98e8b855226e50c54a5d3a543. PrevRequest=061d0a393c5fdf16a0d68541158c9d4a2432ea5b
[INFO] 2023-07-03 16:09:39,424 - Processing RatesCurvesBundles.EUR_EURIBOR_6M done
[INFO] 2023-07-03 16:09:39,425 - Processing RatesCurvesBundles.EUR_XCCY
[INFO] 2023-07-03 16:09:39,426 - Collecting requests
[INFO] 2023-07-03 16:09:42,238 - Requests collected. Request=d3f6d9d360d72c917b9a894b95be71962b17517e. PrevRequest=061d0a393c5fdf16a0d68541158c9d4a2432ea5b
[INFO] 2023-07-03 16:09:43,304 - Processing RatesCurvesBundles.EUR_XCCY done
[INFO] 2023-07-03 16:09:43,305 - Processing RatesCurvesBundles.EUR_RUB
[INFO] 2023-07-03 16:09:43,307 - Collecting requests
[INFO] 2023-07-03 16:09:46,243 - Requests collected. Request=09d40c0940

[INFO] 2023-07-03 16:15:13,594 - Unnecessary info deleted. Job is done
[INFO] 2023-07-03 16:16:41,845 - Result id is fdd6a7bfa6e8e529309f502b405bc671162aeee0


In [5]:
mas_cpties=[]
mas_pnl_explains=[]
mas_total_diff=[]
for el in b:
    mas_cpties.append(ps.get(el).Request['_Counterparty'])
    mas_pnl_explains.append(ps.get(el)._id)
    mas_total_diff.append(ps.get(el).BCVA['Total'])
    
df=pd.DataFrame({'Cpty':mas_cpties,'Explain_hash':mas_pnl_explains,'Total_diff': mas_total_diff})

newoldcpties=pd.read_excel('OldNewCpties.xlsx')
newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
dfotv=df.merge(newoldcpties,on='Cpty',how='left')
dfotv.to_excel(str(datetime.datetime.now().date())+"PnLExplainEdinBase" + dttek+"--"+dtprev+".xlsx")
dfotv.sort_values(by='Total_diff', ascending=False)

,Cpty,Explain_hash,Total_diff,Old/New,Metric
23,VNEKB,34da4f1b08d232189e6d6f0b673081d12ff0401c,"258,876.35",Old,BCVA
555,DTMIR,f56491c34f8e733bd44a5e648cca13dd2f4e6e87,"73,891.08",New,BCVA
662,EPSLL,c405c53f36f5c18fbd7f435430085b80c7a3c52d,"60,112.11",New,BCVA
1131,SZSUB,ac589aaf1c4bc12f2bdfc460f9a13bffccf60eab,"35,168.75",New,BCVA
713,SZNPV,8e0fc08dcda87db90134a0bfdd527af86face1ab,"27,133.05",New,BCVA
...,...,...,...,...,...
454,POLU,f986300e67f5a2e0d6c5e045df144320ee0e3032,"-110,893.56",New,BCVA
613,GRIDD,357cf6af81f1e490037f79db913b198bd5602768,"-146,620.79",New,BCVA
1332,SIBRH,086e547753f452c6b4980b71889fb5c72fc51423,"-171,610.75",New,BCVA
891,SZSTP,b377344788aee7646b6052142b4822b1eedcda37,"-363,967.93",New,BCVA


In [6]:
print("Total diff=",dfotv['Total_diff'].sum())
print("Old diff=",dfotv[dfotv['Old/New']=='Old']['Total_diff'].sum())
print("New diff=",dfotv[dfotv['Old/New']=='New']['Total_diff'].sum())

Total diff= -495642.77057088306
Old diff= 127187.85620497826
New diff= -622830.6267758611


In [9]:
dfotv[dfotv['Cpty']=='FRTGS']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
327,FRTGS,149af3aa4537f8de7dc62761355aaf24c273c061,"-186,869.71",Old,BCVA


In [16]:
dfmasked[dfmasked['Cpty']=='AOEPO']

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
719,AOEPO,113fb9c7e32f2da8aeb2ab78b180614326525a0d,"1,246.73",4abc86a45f425ac6dad0a33fe8a848ed979b4aaa,"35,442.06",New,BCVA,"1,246.73","34,195.33","34,195.33"


In [4]:
r=ps.get('8c5fa2503a992c07bcf34dfa4ae72f400b360dd8')
BCVA_prev_with_greeks=r.Result['BCVA']

r=ps.get(r._nonce)
r.Queries=['BCVA']
BCVA_prev_wo_greeks=ps_utils.compute(ps.put(r))['BCVA']

r=ps.get('3cf9f46b4d79a509e4ea8ed8ad298ea1d2225582')
BCVA_tek_with_greeks=r.Result['BCVA']

r=ps.get(r._nonce)
r.Queries=['BCVA']
BCVA_tek_wo_greeks=ps_utils.compute(ps.put(r))['BCVA']

print("BCVA_prev_with_greeks=",BCVA_prev_with_greeks)
print("BCVA_prev_wo_greeks=",BCVA_prev_wo_greeks)
print("BCVA_tek_with_greeks=",BCVA_tek_with_greeks)
print("BCVA_prev_wo_greeks=",BCVA_tek_wo_greeks)
print('diff_with_greeks=',BCVA_tek_with_greeks-BCVA_prev_with_greeks)
print('diff_wo_greeks=',BCVA_tek_wo_greeks-BCVA_prev_wo_greeks)

[INFO] 2023-03-31 10:28:29,998 - Compute:7866deca901500a7c2e65ee8598eef7f46ebe3a5->b1a58241a5b5f27983cc7bcb2f6bf1d235e44425
[INFO] 2023-03-31 10:28:30,033 - Compute:87e40d8e419d4745a0f4ea0d137d907f2922b774->1bac25cc7505de5df64279b4b428fd5ee3b12a54


BCVA_prev_with_greeks= -5973137.87582982
BCVA_prev_wo_greeks= -5968275.535071469
BCVA_tek_with_greeks= -6141979.579227808
BCVA_prev_wo_greeks= -6155145.248670185
diff_with_greeks= -168841.70339798834
diff_wo_greeks= -186869.71359871607


In [6]:
from collections import defaultdict
def populate_dict(resp_hash,tekcpty,otv):
    r=ps.get(resp_hash)
    for el in r['BCVA']:
        if (el=='Total'):
            otv['Explained'][tekcpty]=r['BCVA'][el]
            otv['Explained'].setdefault('Total',0)
            otv['Explained']['Total']+=r['BCVA'][el]
        elif (el=='Unexplained'):
            otv['Unexplained'][tekcpty]=r['BCVA'][el]
            otv['Unexplained'].setdefault('Total',0)
            otv['Unexplained']['Total']+=r['BCVA'][el]
        elif (el=='Product'):
            otv['Product'][tekcpty]=r['BCVA'][el]
            otv['Product'].setdefault('Total',0)
            otv['Product']['Total']+=r['BCVA'][el]
        elif (el=='Model.MarketDataSet.AsOfDate'):
            otv['Carry'][tekcpty]=r['BCVA'][el]
            otv['Carry'].setdefault('Total',0)
            otv['Carry']['Total']+=r['BCVA'][el]
            
        elif (el=='ErrorBoth'):
            otv['ErrorBoth'][tekcpty]=r['BCVA'][el]
            otv['ErrorBoth'].setdefault('Total',0)
            otv['ErrorBoth']['Total']+=r['BCVA'][el]
        elif (el=='ExpiredCounterparty'):
            otv['ExpiredCounterparty'][tekcpty]=r['BCVA'][el]
            otv['ExpiredCounterparty'].setdefault('Total',0)
            otv['ExpiredCounterparty']['Total']+=r['BCVA'][el]
        elif (el=='NewCounterparty'):
            otv['NewCounterparty'][tekcpty]=r['BCVA'][el]
            otv['NewCounterparty'].setdefault('Total',0)
            otv['NewCounterparty']['Total']+=r['BCVA'][el]
        elif (el=='Rating'):
            otv['Rating'][tekcpty]=r['BCVA'][el]
            otv['Rating'].setdefault('Total',0)
            otv['Rating']['Total']+=r['BCVA'][el]
            
        elif (el.startswith('Model.MarketDataSet.CreditCurves')):
            curve=el.split('.')[3]
            if (curve not in otv['CreditCurves']):
                otv['CreditCurves'][curve]=defaultdict(defaultdict)
            otv['CreditCurves'][curve][tekcpty]=r['BCVA'][el]
            otv['CreditCurves'][curve].setdefault('Total',0)
            otv['CreditCurves'][curve]['Total']+=r['BCVA'][el]
            otv['CreditCurves'].setdefault('Total',0)
            otv['CreditCurves']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.GenericScalars')):
            otv['GenericScalars'][tekcpty]=r['BCVA'][el]
            otv['GenericScalars'].setdefault('Total',0)
            otv['GenericScalars']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.Spots')):
            pair=el.split('.')[3][0:6]
            if (pair not in otv['Spots']):
                otv['Spots'][pair]=defaultdict(defaultdict)
            otv['Spots'][pair][tekcpty]=r['BCVA'][el]
            otv['Spots'][pair].setdefault('Total',0)
            otv['Spots'][pair]['Total']+=r['BCVA'][el]
            otv['Spots'].setdefault('Total',0)
            otv['Spots']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.RatesCurvesBundles')):
            ccy=el.split('.')[3][0:3]
            curve=el.split('.')[3]
            if (ccy not in otv['Rates']):
                otv['Rates'][ccy]=defaultdict(defaultdict)
            otv['Rates'][ccy][curve][tekcpty]=r['BCVA'][el]
            otv['Rates'][ccy][curve].setdefault('Total',0)
            otv['Rates'][ccy][curve]['Total']+=r['BCVA'][el]
            otv['Rates'][ccy].setdefault('Total',0)
            otv['Rates'][ccy]['Total']+=r['BCVA'][el]
            otv['Rates'].setdefault('Total',0)
            otv['Rates']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.VolatilitySurfaces')):
            flagIsFXVol=True
            if (el.split('.')[3][3]=='_'):
                flagIsFXVol=False
            if not flagIsFXVol:
                ccy=el.split('.')[3][0:3]
                curve=el.split('.')[3]
                if (ccy not in otv['RatesVols']):
                    otv['RatesVols'][ccy]=defaultdict(defaultdict)
                otv['RatesVols'][ccy][curve][tekcpty]=r['BCVA'][el]
                otv['RatesVols'][ccy][curve].setdefault('Total',0)
                otv['RatesVols'][ccy][curve]['Total']+=r['BCVA'][el]
                otv['RatesVols'][ccy].setdefault('Total',0)
                otv['RatesVols'][ccy]['Total']+=r['BCVA'][el]
                otv['RatesVols'].setdefault('Total',0)
                otv['RatesVols']['Total']+=r['BCVA'][el]
            elif flagIsFXVol:
                pair=el.split('.')[3][0:6]
                if (pair not in otv['FXVols']):
                    otv['FXVols'][pair]=defaultdict(defaultdict)
                otv['FXVols'][pair][tekcpty]=r['BCVA'][el]
                otv['FXVols'][pair].setdefault('Total',0)
                otv['FXVols'][pair]['Total']+=r['BCVA'][el]
                otv['FXVols'].setdefault('Total',0)
                otv['FXVols']['Total']+=r['BCVA'][el]
                
                
            

    return otv

def get_pnl_table(a):
    bcva_vals=[]
    bcva_mas1=[]
    bcva_mas2=[]
    for el in a:
        if el in ['Carry','GenericScalars','Explained','Unexplained','Product']:
            bcva_mas1.append(el)
            bcva_mas2.append(el)
            bcva_vals.append(a[el]['Total'])
        elif el in ['CreditCurves','Rates','Spots','RatesVols','FXVols']:
            for k in a[el]:
                if k=='Total':
                    continue
                bcva_mas1.append(el)
                bcva_mas2.append(k)
                bcva_vals.append(a[el][k]['Total'])
        else:
            print("Bad el:",el)
    bcva_arrays = [np.array(bcva_mas1),np.array(bcva_mas2)]
    bcva_df=pd.DataFrame(bcva_vals, index=bcva_arrays)
    return bcva_df

In [7]:

dfotv_old=dfotv[dfotv['Old/New']=='Old'].reset_index(drop=True)
dfotv_new=dfotv[dfotv['Old/New']=='New'].reset_index(drop=True)

NAME=dfotv_old
otv=defaultdict(defaultdict)
for i in range(len(NAME)):
    otv=populate_dict(NAME['Explain_hash'][i],NAME['Cpty'][i],otv)
for el in otv:
    print(el,otv[el]['Total'])
    
    
t=get_pnl_table(otv)
print("Sum of really explained (wo unexplained)=",t.values.sum()-t.loc['Explained'].values-t.loc['Unexplained'].values)
#print("Explained=",t.loc['Explained'].values)
#print("Actual diff=",NAME['diff_BCVA'].sum())
t.to_excel(str(datetime.datetime.now().date())+"OldBookTable" + dttek+"--"+dtprev+".xlsx")
t

Explained 306348.476162335
Carry 101299.95510272522
CreditCurves -44.302705900321264
Rates 27269.365939974487
Spots 346692.92614100414
RatesVols -152664.48342299595
Unexplained -4130.510232400651
Product -12047.460527824062
FXVols -27.014132247220857
Sum of really explained (wo unexplained)= [[310478.98639474]]


0
Explained    Explained            306,348.48
Carry        Carry                101,299.96
CreditCurves RUB_SBER_OFZ_SNR_CR      -50.35
             RUB_MINFIN_OFZ             6.42
             USD_KAZKH_SNR_CR          -0.37
Rates        RUB                   36,174.21
             USD                     -787.77
             CNH                   -8,119.59
             EUR                     -117.86
             XAU                      -73.79
             JPY                        0.00
             CHF                      194.17
Spots        USDRUB                -2,471.25
             CNHRUB               297,070.88
             EURRUB                28,599.84
             XAURUB                -8,087.15
             JPYRUB                     0.00
             CHFRUB                31,580.60
RatesVols    RUB                 -152,664.48
             USD                        0.00
             EUR                        0.00
Unexplained  Unexplained           -4,130.51
Product      Product              -12,047.46
FXVols       USDRUB                     0.00
             CNHRUB                   -27.01
             EURRUB                     0.00
             JPYRUB                     0.00
             CHFRUB                     0.00

In [8]:
otv['Product']

defaultdict(None,
            {'ELKOR': -3.495413623020795,
             'Total': -12047.460527824062,
             'ATMEN': 2742.665187102157,
             'VERMO': -1653.4311194951433,
             'RTRTR': 0.0,
             'VUBOB': 6021.069015666932,
             'ZELRO': -41.41202638984214,
             'RUSKZ': -12.820471529052554,
             'NPPKV': -0.454700993932172,
             'OLOSS': -1.4654096502218783,
             'SERVK': 0.0,
             'SRTNN': 0.0,
             'PUCKO': -379.6368303609124,
             'INTSL': 3.1749704928504343,
             'LKTIT': 0.0,
             'SIBSI': -107.70181787680895,
             'SZDIN': -20.026436814173394,
             'SRBKR': 2140.5727053626188,
             'TDMMM': -1133.45883904797,
             'MTLLK': -3.585188111671795,
             'METRZ': -3803.6025907414264,
             'ANKRR': -71.93248883172475,
             'CVETA': 356.17004855369396,
             'SZGRO': 1259.4964047346975,
             'TERRI': -20.2779

In [9]:

dfotv_old=dfotv[dfotv['Old/New']=='Old'].reset_index(drop=True)
dfotv_new=dfotv[dfotv['Old/New']=='New'].reset_index(drop=True)

NAME=dfotv_new
otv=defaultdict(defaultdict)
for i in range(len(NAME)):
    otv=populate_dict(NAME['Explain_hash'][i],NAME['Cpty'][i],otv)
for el in otv:
    print(el,otv[el]['Total'])
    
    
t=get_pnl_table(otv)
print("Sum of really explained (wo unexplained)=",t.values.sum()-t.loc['Explained'].values-t.loc['Unexplained'].values)
#print("Explained=",t.loc['Explained'].values)
#print("Actual diff=",NAME['diff_BCVA'].sum())
t.to_excel(str(datetime.datetime.now().date())+"NewBookTable" + dttek+"--"+dtprev+".xlsx")
t

Explained -1402600.2202305335
Product -1506174.5036731013
Carry 47494.17538390769
CreditCurves 110.51823450733997
Rates 3107.8603782783116
Spots 1218.1828432223172
RatesVols -4819.0638030524615
Unexplained 56471.42235034114
FXVols -8.811944637044832
Sum of really explained (wo unexplained)= [[-1459071.64258088]]


0
Explained    Explained           -1,402,600.22
Product      Product             -1,506,174.50
Carry        Carry                   47,494.18
CreditCurves RUB_SBER_OFZ_SNR_CR         -3.74
             RUB_MINFIN_OFZ             114.26
Rates        RUB                     -9,534.19
             USD                      1,435.16
             CNH                     11,941.31
             EUR                       -171.18
             XAU                       -563.24
Spots        USDRUB                 360,943.87
             CNHRUB                -708,263.54
             EURRUB                 352,435.28
             XAURUB                  -3,897.43
RatesVols    RUB                     -4,819.06
             USD                          0.00
             EUR                          0.00
Unexplained  Unexplained             56,471.42
FXVols       CNHRUB                      -8.81
             USDRUB                       0.00
             EURRUB                      -0.00

In [10]:
otv['Product']

defaultdict(None,
            {'1AAMG': 8.73758613486369,
             'Total': -1506174.5036731013,
             'SZSMM': 1.294008688424694,
             'REMKS': 10.33272988518857,
             'SZPPB': 0.0,
             'MSGRP': -19990.656968139985,
             'SZNKK': -145.2963104865339,
             'NIMZK': 121.10095687562624,
             'MLKPK': 331.90938998507363,
             'ZONDX': 53.953737064808266,
             'SZPRS': -10210.14435465636,
             'SZKOT': -12.874172799284679,
             'MELSA': 2.409562016026902,
             'MBKRK': -701005.0215828404,
             'SZDED': -4064.5037618183133,
             'KLINN': 4.321444816926132,
             'MZTON': 0.0,
             'PVBMV': -6057.466429022832,
             'TVGRR': 2.0353419181923478,
             'JDKEN': 0.0,
             'TTEHH': 4.010131319730704,
             'GLOFA': -1965.7168325052555,
             'PRIVP': 3.428568549546071,
             'GKCTT': -1036.7823630802013,
             '1AALN':

In [11]:
dfotv_old[dfotv_old['Cpty']=='KARDL']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
585,KARDL,6c0638bbdedb607755532fe030a653ec92b9b28c,"-28,735.74",Old,BCVA


In [12]:
dfotv_new[dfotv_new['Cpty']=='SIBRH']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
576,SIBRH,086e547753f452c6b4980b71889fb5c72fc51423,"-171,610.75",New,BCVA


In [13]:
m=0
for el in dfotv_old['Explain_hash']:
    el=ps.get(el)
    try:
        m+=el['BCVA']['Rating']
    except:
        continue
m

0

In [14]:
m=0
for el in dfotv_new['Explain_hash']:
    el=ps.get(el)
    try:
        m+=el['BCVA']['Rating']
    except:
        continue
m

0

In [17]:
s1=set()
r=ps.get('65c171951c71cee1f774b10e3396231db03d074b')
for el in r.Products:
    s1.add(el['_ContractID'])
s2=set()
r=ps.get('316a8c84ede11278b1d42d7e872a3737aba20ad9')
for el in r.Products:
    s2.add(el['_ContractID'])
print(len(s1))
print(len(s2))
print(s1.difference(s2))
print(s2.difference(s1))

16
14
{'38392628', '38392640'}
set()


In [19]:
len(s2)

117

In [22]:
r=ps.get('4886dbdb4b0fae6aa5f9fcf529843214f6b242c3')
s24=set()
for el in r.Products:
    s24.add(el._id)
s24

{'05ffd67d562e1cb25a46ea8f0494beabc38521b5',
 '1f008a983e9d48fc591af28b07d50e8e35a1c01a',
 '263af42506e0504c5a6308c3feac69ee453bb391',
 '2cf60df268ba4263e2f392fac7b1e7e161aebf4d',
 '2fff5c380cfcd684abc84bc6183e74f1be9df49a',
 '39c2efff94df0c5d7cc9da36e309a6a0bf603df7',
 '3dec782bdf1035ef340d7b2bfef657b37988c80c',
 '690cd72c1bfb54f736cf5592097744dc2ea1b3ed',
 '72324f5cdb3fa4863bb2dc6b2a9175de0f40cc0c',
 '8eeb4ac8f815016ca1a67203e60b980b783adb33',
 '9d58781a594705aa053f6a39a239f098af1a3b46',
 'ec73d26cd3bd259fc3e12d8564ac84d99b61d1c8'}

In [23]:
r=ps.get('6439fe9b20c630f2a8aae56e875de3be1b3286f8')
s25=set()
for el in r.Products:
    s25.add(el._id)
s25

{'05ffd67d562e1cb25a46ea8f0494beabc38521b5',
 '1f008a983e9d48fc591af28b07d50e8e35a1c01a',
 '263af42506e0504c5a6308c3feac69ee453bb391',
 '2cf60df268ba4263e2f392fac7b1e7e161aebf4d',
 '2fff5c380cfcd684abc84bc6183e74f1be9df49a',
 '3dec782bdf1035ef340d7b2bfef657b37988c80c',
 '690cd72c1bfb54f736cf5592097744dc2ea1b3ed',
 '72324f5cdb3fa4863bb2dc6b2a9175de0f40cc0c',
 '8bb8c2f288ee2afdfb47010dc162ccd6d1062f7d',
 '91b402ad7461c6087121e6f9d43ea818e154b6fd',
 '9d58781a594705aa053f6a39a239f098af1a3b46',
 'ec73d26cd3bd259fc3e12d8564ac84d99b61d1c8'}

In [24]:
s25.difference(s24)

{'8bb8c2f288ee2afdfb47010dc162ccd6d1062f7d',
 '91b402ad7461c6087121e6f9d43ea818e154b6fd'}

In [24]:
#mas=[]
ps_utils.switch_to_dev()
r=ps.get('8f97fbd1299f6f5632affb0be6646df5fb6113b8')
for el in r.Result:
    try:
        s=el.StatusText
    except:
        s='None'
    if (s!='done'):
        print(ps.get(el._nonce)['_Counterparty'], el._id,s[0:65])
        #mas.append(ps.get(el._nonce)._id)
#mas

[INFO] 2023-06-08 12:15:51,242 - ps client switched from prod_pvlss to dev


AVTTT 794289296f4a79d70201ea2fca47b0e02d9c87e6 signal: floating point exception (core dumped)
RPZSO 4ff80e75ea6e15ab82a2cd3ad35f9da8063ee475 Repeated failures (5), last error: signal: floating point excepti
SPPNA 71bd1ab3c3893b730b110baa86ad0857cbe29829 signal: floating point exception (core dumped)
PTROL 2bc05a93e0188321dcd83784fa4864d3753df4e0 Unable to find index AI95K5_UFM_SPX_AVG in the static data set. w
GODOV 17f7f536db75017cb0c07c37cdc5434d7d0e7fd1 None
FSKSS dedd6d628ac19c17a795b3ebce2279f32b5c0dd0 Unable to find index RU_CPI_2000 in the static data set. while co
MBDDR 9a1f29177a165b88166cbe65ef2ed570477c637c Unable to find index INRRUB in the static data set. while constru
ISTRA 1bba957c31adff693dfd561f9463eec54782bb63 signal: floating point exception (core dumped)
MASKL 122c55c49e042bf5bf84b82d60d92d54ee7c5485 signal: floating point exception (core dumped)
KIRAN 26e4edee95006eee4c94724da3a63230998989ef signal: floating point exception (core dumped)
EXTRE 98a0ecf61bdc4fed467c

In [49]:
22260*0.05

1113.0

In [18]:
r.Model.MarketDataSet.Spots['EURRUB']

NameError: name 'r' is not defined

In [35]:
r=ps.get('1329ece514cc5041e3d1a34fc33aa36ea2cc6101')
#r.Product.NettingSets[0].MarginSets[1]['CSA']='123fb52aa9748c8cf42f5c4e96dffbb1603088fe'
ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-06-23 11:39:31,558 - Compute:1329ece514cc5041e3d1a34fc33aa36ea2cc6101->40fd924b5e09f994d402d81e038fd963f84be949


Dict([('BCVA', 342256.4178827792),
      ('CVA', -27360.073004739785),
      ('DVA', 369616.490887519),
      ('ProductValue', -10508009.95689506),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 49702.90608955732),
      ('CashToSettleToday', 0.0)])

In [34]:
r=ps.get('1329ece514cc5041e3d1a34fc33aa36ea2cc6101')
r.Product.NettingSets[0].MarginSets[1]['CSA']='123fb52aa9748c8cf42f5c4e96dffbb1603088fe'
ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-06-23 11:38:57,978 - Compute:64193523d047367621d6f49a96893d91f980bde2->7954c2ec500a26a806001df2286baadbd95234b9


Dict([('BCVA', 157750.83596511622),
      ('CVA', -69582.82411569265),
      ('DVA', 227333.66008080886),
      ('ProductValue', -10508009.95689506),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 49702.90608955732),
      ('CashToSettleToday', 0.0)])